In [1]:
import pandas as pd
import numpy as np
from scipy.sparse import lil_matrix

In [2]:
#df_tr_num = pd.read_hdf('input_hdf/train_numeric_928cols.hdf5','table')
df_tr_cat = pd.read_hdf('input_hdf/train_categorical_205cols.hdf5','table')
df_te_cat = pd.read_hdf('input_hdf/test_categorical_205cols.hdf5','table')
#df_tr_date = pd.read_hdf('input_hdf/train_date_162cols.hdf5','table')
#df_tr_cat = df_tr_cat.astype(np.object)

In [3]:
df_cat = pd.concat([df_tr_cat, df_te_cat], ignore_index=True)

In [4]:
indd = df_cat.Id.astype(np.int32)

In [5]:
df_cat.drop(['Id'],axis=1,inplace=True)

In [ ]:
## let's wrap that in a function for re-use 
def cleanup_data(df, cutoffPercent = .01):
    for col in df:
        sizes = df[col].value_counts(normalize = True)
        # get the names of the levels that make up less than 1% of the dataset
        values_to_delete = sizes[sizes<cutoffPercent].index
        df[col].ix[df[col].isin(values_to_delete)] = "Other"
    return df
##
## let's also wrap this into a function.
def get_binary_values(data_frame):
    """encodes cateogrical features in Pandas.
    """
    all_columns = pd.DataFrame( index = data_frame.index)
    for col in data_frame.columns:
        data = pd.get_dummies(data_frame[col], prefix=col, sparse=True, drop_first=True)
        all_columns = pd.concat([all_columns, data], axis=1)
    return all_columns

def sparse_df_to_array(df):
    """ Convert sparse dataframe to sparse array csr_matrix used by
    scikit learn. """
    arr = lil_matrix(df.shape, dtype=np.int8)
    for i, col in enumerate(df.columns):
        ix = df[col] != 0
        arr[np.where(ix), i] = df.ix[ix, col]

    return arr.tocsr()

In [ ]:
ohe_encoded_sparse_df = get_binary_values(df_cat)

In [ ]:
ohe_encoded_sparse_df.info()

<class 'pandas.sparse.frame.SparseDataFrame'>
RangeIndex: 2367495 entries, 0 to 2367494
Columns: 656 entries, L0_S1_F27_1 to L3_S49_F4220_3
dtypes: float64(656)
memory usage: 264.0 MB


In [ ]:
ohe_encoded_sparse_df.head()

,L0_S1_F27_1,L0_S1_F27_2,L0_S1_F29_1,L0_S1_F31_1,L0_S1_F31_2,L0_S1_F31_3,L0_S1_F31_4,L0_S1_F31_5,L0_S1_F31_6,L0_S2_F33_1,...,L3_S43_F4084_3,L3_S43_F4084_4,L3_S44_F4102_1,L3_S46_F4134_1,L3_S47_F4141_1,L3_S49_F4207_1,L3_S49_F4217_1,L3_S49_F4220_1,L3_S49_F4220_2,L3_S49_F4220_3
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
df_all_csr = sparse_df_to_array(ohe_encoded_sparse_df)

In [ ]:
df_all_csr

<2367495x656 sparse matrix of type '<class 'numpy.int8'>'
	with 34606599 stored elements in Compressed Sparse Row format>

In [ ]:
colss = ohe_encoded_sparse_df.columns
df_all_dense = pd.DataFrame(df_all_csr.todense(),columns=colss)
df_all_dense.insert(0, 'Id', indd)

In [ ]:
type(df_all_dense)

pandas.core.frame.DataFrame

In [ ]:
df_all_dense.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2367495 entries, 0 to 2367494
Columns: 657 entries, Id to L3_S49_F4220_3
dtypes: int32(1), int8(656)
memory usage: 1.5 GB


In [ ]:
df_all_dense.head()

,Id,L0_S1_F27_1,L0_S1_F27_2,L0_S1_F29_1,L0_S1_F31_1,L0_S1_F31_2,L0_S1_F31_3,L0_S1_F31_4,L0_S1_F31_5,L0_S1_F31_6,...,L3_S43_F4084_3,L3_S43_F4084_4,L3_S44_F4102_1,L3_S46_F4134_1,L3_S47_F4141_1,L3_S49_F4207_1,L3_S49_F4217_1,L3_S49_F4220_1,L3_S49_F4220_2,L3_S49_F4220_3
0,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,11,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
df_all_dense.to_hdf('df_all_dense.hdf5','table')

In [21]:
tr_ind = np.arange(0,df_tr_cat.shape[0])
te_ind = np.arange(df_tr_cat.shape[0],df_cat.shape[0])
df_tr_dense = df_all_dense.ix[tr_ind]
df_te_dense = df_all_dense.ix[te_ind]

In [26]:
#now, let's find features with no variance 
toKeep = []
toDelete = []
## loop through the DataFrame's columns
for col in df_tr_dense:
    ## if the value_counts method returns more than one uniqe entity,
    ## append the column name to 'toKeep'
    if len(df_tr_dense[col].value_counts()) > 1:
        toKeep.append(col)
    ## if not, append to 'toDelete'.
    else:
        toDelete.append(col)

In [27]:
df_tr_dense.drop(toDelete, 1, inplace = True)
df_te_dense.drop(toDelete, 1, inplace = True)

In [28]:
df_tr_dense.shape,df_te_dense.shape

((1183747, 596), (1183748, 596))

In [30]:
df_tr_dense.to_hdf('train_categorical_205cols_596_OHE.hdf5','table')
df_te_dense.to_hdf('test_categorical_205cols_596_OHE.hdf5','table')